In [ ]:
import cv2
vidcap = cv2.VideoCapture('/content/S22_3.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  #print('Read a new frame: ', success)
  count += 1 
totalframecount= int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print("The total number of frames in this video is ", totalframecount)


In [ ]:
import os
import argparse
import time
!pip install mxnet
!pip install gluoncv

In [ ]:
import numpy as np
import mxnet as mx
from mxnet.gluon.data.vision import transforms
import gluoncv
# using cpu
ctx = mx.cpu(0)
model = gluoncv.model_zoo.get_model('monodepth2_resnet18_kitti_stereo_640x192',pretrained_base=False, ctx=ctx, pretrained=True)


In [ ]:
import PIL.Image as pil
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
count = 0
while count<totalframecount:
  filename="frame"+str(count)+".jpg"  
  img = pil.open(filename).convert('RGB')
  #plt.imshow(img)
  #plt.show()
  original_width, original_height = img.size
  feed_height = 192
  feed_width = 640

  img = img.resize((feed_width, feed_height), pil.LANCZOS)
  img = transforms.ToTensor()(mx.nd.array(img)).expand_dims(0).as_in_context(context=ctx)
  outputs = model.predict(img)
  disp = outputs[("disp", 0)]
  disp_resized = mx.nd.contrib.BilinearResize2D(disp, height=original_height, width=original_width)

  disp_resized_np = disp_resized.squeeze().as_in_context(mx.cpu()).asnumpy()
  vmax = np.percentile(disp_resized_np, 95)
  normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
  mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
  colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
  im = pil.fromarray(colormapped_im)
  im.save('test_output'+str(count)+'.png')

  import matplotlib.image as mpimg
  disp_map = mpimg.imread('test_output'+str(count)+'.png')
  #plt.imshow(disp_map)
  #plt.show()
  count += 1

In [ ]:
import cv2
import numpy as np

img=[]
for i in range(0,totalframecount):
    img.append(cv2.imread("test_output"+str(i)+'.png'))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height,width,layers=img[1].shape 
#video=cv2.VideoWriter("video3.avi",fourcc,55,(width,height))
video=cv2.VideoWriter("video3.mp4",fourcc,55,(width,height))

for j in range(0,totalframecount):
    video.write(img[j])

cv2.destroyAllWindows()
video.release()